# Code for processing Lagrangian dFAD metrics
#### Output
This code processes the raw output of the parcels simulations (>20GB) into several metrics.
These metrics are subsequently stored in .nc files with the shape (longitude, latitude, drop_time), which provides the monthly average of each metric per drop location. The mean value is calculated and stored in a separate outputfile.

The calculated metrics are:
- Displacement (dist_list)
- Travel distance (dist_trav_list)
- Distance ratio (eddy_list)
- loopiness (loop_list)

#### Code structure
The code is roughly structured in 3 parts, which are divided over multiple cells:
1) loading data set
2) calculating metrics
3) output metrics

In [1]:
#import modules
import xarray as xr
import numpy as np
from datetime import timedelta as delta
import pandas as pd

In [2]:
#Dataset parameters
path = "C:/Users/FrankemollePFVW/Downloads/dFAD_data/"
particletype = 'surface' # choose either 'surface' or 'depth' to load different datasets

xmin,xmax = 140, 285 # minimum/maximum longitude (default = 140, 285)
ymin,ymax = -15, 15  # minimum/maximum latitude  (default = -15, 15)
end_time = 30        # particle travel time (default = 30 days)
lenX,lenY = 291,61   # gridsize in longitudinal, latitudinal direction (default = 291,61)

data =xr.open_zarr(path + '20062021' + particletype + '_wholepacific.zarr')

lat = data['lat'].values
#lat = np.delete(lat, np.s_[end_time:-1], 1)
#lon = data['lon'].values
#lon = np.delete(lon, np.s_[end_time:-1], 1)
#time = data['time'].values.astype('datetime64[D]')
#t = np.arange('2006-11', '2022-01', dtype='datetime64[D]') #runtime of dataset
#t = t[:int(M*10):10] #particle release dates

In [12]:

np.sum(lat[:,0] > -90)


9834054

In [3]:
#Creating the particle grid (unneccesary?)
lonEEZ = np.linspace(xmin,xmax,num = lenX)
latEEZ = np.linspace(ymin,ymax, num = lenY)
latlist,lonlist = np.reshape(latEEZ,-1),np.reshape(lonEEZ,-1)
lonEEZ = (lonEEZ[1:] + lonEEZ[:-1])/2 #has to be in center of gridboxes...
latEEZ = (latEEZ[1:] + latEEZ[:-1])/2

#lon[lon<0] = lon[lon<0]+360

#lat_i_a=np.digitize(lat,latEEZ)
#lon_i_a=np.digitize(lon,lonEEZ)

In [4]:
#Displacement distance
loc = np.array([lon[:,0],lat[:,0],lon[:,end_time-1],lat[:,end_time-1]])
loc[loc>180] = loc[loc>180] - 360
loc = loc*np.pi/180

dlon = loc[0,:] - loc[2,:]
dlat = loc[1,:] - loc[3,:]

a = np.sin(dlat/2)**2 + np.cos(loc[1,:]) * np.cos(loc[3,:]) * np.sin(dlon/2)**2
c = 2 * np.arcsin(np.sqrt(a)) 
r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
dist_list = c*r

In [5]:
#total travel distance
dist_trav_list = np.zeros(len(lon)) #create empty array of zeros
for i in range(end_time-1): 
    dlon = (lon[:,i] - lon[:,i+1])*np.pi/180
    dlat = (lat[:,i] - lat[:,i+1])*np.pi/180
    a = np.sin(dlat/2)**2 + np.cos(lat[:,i+1]*np.pi/180) * np.cos(lat[:,i]*np.pi/180) * np.sin(dlon/2)**2

    c = 2 * np.arcsin(np.sqrt(a)) 
    dist_trav_list += c*r

#Free up some memory by deleting variables    
#del a
#del dlon
#del dlat
#del c

In [6]:
# distance ratio
eddy_list = dist_list/dist_trav_list

C:\Users\FrankemollePFVW\AppData\Local\Temp\ipykernel_15964\1476333259.py:2: RuntimeWarning: invalid value encountered in divide
  eddy_list = dist_list/dist_trav_list


In [7]:
# loopiness
#Formula for Bearing (angle between two points)
#according to: https://www.igismap.com/formula-to-find-bearing-or-heading-angle-between-two-points-latitude-longitude/
dlonloop = (lon[:,1:end_time] - lon[:,:end_time-1])*np.pi/180

Xloop = np.cos(lat[:,1:end_time]*np.pi/180)*np.sin(dlonloop)
Yloop = np.cos(lat[:,:end_time-1]*np.pi/180)*np.sin(lat[:,1:end_time]*np.pi/180) -  np.cos(lat[:,1:end_time]*np.pi/180)*np.sin(lat[:,:end_time-1]*np.pi/180)*np.cos(dlonloop)
del dlonloop

bear_list = np.arctan2(Xloop,Yloop) #list of bearings between each point 
del Xloop
del Yloop
#calculate angle differences from bearings
anglediff_list = bear_list[:,1:] - bear_list[:,:-1]
#change the domain of values to [+pi,-pi]
anglediff_list[anglediff_list > np.pi] += - 2*np.pi
anglediff_list[anglediff_list <= -np.pi] +=  2*np.pi

#Find loopiness by taking the sum of all angle differences
loop_list = np.sum(anglediff_list, axis = 1)

#del bear_list
#del anglediff_list

In [8]:
M= int(len(lon[:,0])/(lenY*lenX) - 18) #amount of particle releases in the data
N= int(lenY*lenX)                      #amount of particles per release time
#del lat
#del lon

#Rearranging the dataset to [timeseries of metric, drop location]
dist_cell = np.zeros([M,N])
dist_trav_cell = np.zeros([M,N])
eddy_cell = np.zeros([M,N])
loop_cell = np.zeros([M,N])

for i in range(N):
    dist_cell[:,i]      = dist_list[i:-N*18:N]
    dist_trav_cell[:,i] = dist_trav_list[i:-N*18:N]
    eddy_cell[:,i]      = eddy_list[i:-N*18:N]
    loop_cell[:,i]      = loop_list[i:-N*18:N]

del dist_list
del dist_trav_list
del eddy_list
del loop_list

#transform back statistic = np.reshape(statanalysis[:,0], (lenX,lenY)) how to transform 


In [9]:
t = np.arange('2006-11', '2022-01', dtype='datetime64[D]') #runtime of dataset
t = t[:int(M*10):10] #particle release dates
dist = xr.Dataset(
    {"displacement": (("releasetime", "loc"), dist_cell)},
    coords={
        "releasetime": t,
        "loc": np.linspace(1,N,num=N),
        #"z": ("x", list("abcd")),
    },
)


dist.to_netcdf("dFAD_displacement_" + particletype + "_raw.nc")

In [10]:
dist_trav = xr.Dataset(
    {"traveldistance": (("releasetime", "loc"), dist_trav_cell)},
    coords={
        "releasetime": t,
        "loc": np.linspace(1,N,num=N),
        #"z": ("x", list("abcd")),
    },
)
dist_trav.to_netcdf("dFAD_traveldistance_" + particletype + "_raw.nc")

In [11]:
eddy = xr.Dataset(
    {"distanceratio": (("releasetime", "loc"), eddy_cell)},
    coords={
        "releasetime": t,
        "loc": np.linspace(1,N,num=N),
        #"z": ("x", list("abcd")),
    },
)
eddy.to_netcdf("dFAD_distanceratio_" + particletype + "_raw.nc")

In [12]:
loop = xr.Dataset(
    {"loopiness": (("releasetime", "loc"), loop_cell)},
    coords={
        "releasetime": t,
        "loc": np.linspace(1,N,num=N),
        #"z": ("x", list("abcd")),
    },
)
loop.to_netcdf("dFAD_loopiness_" + particletype + "_raw.nc")

In [13]:
del lat
del lon